In [1]:
# ====================================================
# CFG
# ====================================================
import numpy as np
class CFG:
    wandb = True
    DEBUG = False
    TO_KAGGLE = True
    score_path = "/home/jupyter/output/scores/scores003.csv"
    MEMO = "ベースライン"
    file_name = "001"
    model="funnel/intermediate"
    n_fold=4
    trn_fold=[0,1,2,3]
    model_config_path = f"/home/jupyter/models/{model}/"
    model_bin_path = f"/home/jupyter/models/{model}/"
    competition='FB3'
    apex=True
    print_freq=20
    num_workers=4
    gradient_checkpointing=False
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=6
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=8
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    train=True
    
if CFG.DEBUG:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [2]:
import os
import datetime
import pickle
import glob

# ====================================================
# datetime
# ====================================================
t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
now = datetime.datetime.now(JST)
date = now.strftime('%Y%m%d')
date2 = now.strftime('%Y%m%d%H%M')


# ====================================================
# file_path
# ====================================================
if "/" in CFG.model:
    model_name = CFG.model.replace("/","-")
else:
    model_name = CFG.model

path ="/home/jupyter/feedback-prize-english-language-learning/"
if CFG.DEBUG:
    OUTPUT_DIR = f'/home/jupyter/output/ex/DEBUG/{model_name}/{CFG.file_name}/{date2}/'
else:
    OUTPUT_DIR = f'/home/jupyter/output/ex/{model_name}/{CFG.file_name}/{date2}/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)



# ====================================================
# wandb 
# ====================================================
if CFG.wandb:
    import wandb
    
    def class2dict(f):
            return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))
        
    project='Feedback Prize - English Language Learning'
    if CFG.DEBUG:
        group = "DEBUG"
    else:
        group = model_name
    wandb_name = f"{CFG.file_name}_{date2}"
    job_type = CFG.file_name  #"train"

    wandb_api = "your_id"
    wandb.login(key=wandb_api)
    anony = None
    run = wandb.init(project=project, 
                         name = wandb_name,
                         config=class2dict(CFG),
                         group=group,
                         job_type=job_type,
                         anonymous=anony)

wandb: Currently logged in as: hiroki8383. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Library

In [3]:
# !pip download transformers==4.21.2
# !pip download tokenizers==0.12.1

In [4]:
# !pip install transformers
# !pip install tokenizers

In [5]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
import shutil
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

os.system('pip install iterative-stratification==0.1.7')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=/home/jupyter/code_baseline/FB3_pip_wheels transformers')
os.system('python -m pip install --no-index --find-links=/home/jupyter/code_baseline/FB3_pip_wheels tokenizers')
import tokenizers
import transformers
# print(f"tokenizers.__version__: {tokenizers.__version__}")
# print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Found existing installation: transformers 4.21.2
Uninstalling transformers-4.21.2:
  Successfully uninstalled transformers-4.21.2


Found existing installation: tokenizers 0.12.1
Uninstalling tokenizers-0.12.1:
  Successfully uninstalled tokenizers-0.12.1


Looking in links: /home/jupyter/code_baseline/FB3_pip_wheels
Processing /home/jupyter/code_baseline/FB3_pip_wheels/transformers-4.21.2-py3-none-any.whl
Processing /home/jupyter/code_baseline/FB3_pip_wheels/tokenizers-0.12.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.1 requires transformers<4.21,>=4.1, but you have transformers 4.21.2 which is incompatible.


Looking in links: /home/jupyter/code_baseline/FB3_pip_wheels


env: TOKENIZERS_PARALLELISM=true


# Utils

In [6]:
# ====================================================
# Utils
# ====================================================
def MCRMSE(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores


def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

# Data Loading

In [7]:
# ====================================================
# Data Loading
# ====================================================

train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
submission = pd.read_csv(path+'sample_submission.csv')

Fold = MultilabelStratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(train, train[CFG.target_cols])):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)

if CFG.DEBUG:
    # display(train.groupby('fold').size())
    train = train.sample(n=50, random_state=0).reset_index(drop=True)
    # display(train.groupby('fold').size())

# tokenizer

In [8]:
# ====================================================
# tokenizer models/roberta/roberta-base/config.json
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(f"/home/jupyter/models/{CFG.model}/")
CFG.tokenizer = tokenizer

# Dataset

In [9]:
# ====================================================
# Define max_len
# ====================================================
# lengths = []
# tk0 = tqdm(train['full_text'].fillna("").values, total=len(train))
# for text in tk0:
#     length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
#     lengths.append(length)
# CFG.max_len = max(lengths) + 3 # cls & sep & sep
# LOGGER.info(f"max_len: {CFG.max_len}")

In [10]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer.encode_plus(
        text, 
        return_tensors=None, 
        add_special_tokens=True, 
        max_length=CFG.max_len,
        pad_to_max_length=True,
        truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['full_text'].values
        self.labels = df[cfg.target_cols].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        label = torch.tensor(self.labels[item], dtype=torch.float)
        return inputs, label
    

def collate(inputs):
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:,:mask_len]
    return inputs

# Model

In [11]:
# ====================================================
# Model
# ====================================================
#MeanPoolingはoutput_hidden_statesに関係している   https://qiita.com/niship2/items/f84751aed893da869cec
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings
    

class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model_config_path, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model_bin_path, config=self.config)
        else:
            self.model = AutoModel(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()
        self.pool = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, 6)
        self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        feature = self.pool(last_hidden_states, inputs['attention_mask'])
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(feature)
        return output

In [12]:
# model_config_path = f"/home/jupyter/models/deberta/{model}/model"
# model_bin_path = f"/home/jupyter/models/deberta/{CFG.model}/model"

In [13]:
# conf = AutoConfig.from_pretrained(f"/home/jupyter/models/deberta/{CFG.model}/model", output_hidden_states=True)
# AutoModel.from_pretrained(f"/home/jupyter/models/deberta/{CFG.model}/model", config=conf)

# Loss

In [14]:
# ====================================================
# Loss
# ====================================================
class RMSELoss(nn.Module):
    def __init__(self, reduction='mean', eps=1e-9):
        super().__init__()
        self.mse = nn.MSELoss(reduction='none')
        self.reduction = reduction
        self.eps = eps

    def forward(self, y_pred, y_true):
        loss = torch.sqrt(self.mse(y_pred, y_true) + self.eps)
        if self.reduction == 'none':
            loss = loss
        elif self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()
        return loss

# Helpler functions

In [15]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (inputs, labels) in enumerate(train_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs)
            loss = criterion(y_preds, labels)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
        if CFG.wandb:
            wandb.log({f"[fold{fold}] loss": losses.val,
                       f"[fold{fold}] lr": scheduler.get_lr()[0]})
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(inputs)
            loss = criterion(y_preds, labels)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

# train loop

In [16]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_cols].values
    
    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size * 2,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True)
    torch.save(model.config, OUTPUT_DIR+'config.pth')
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.SmoothL1Loss(reduction='mean') # RMSELoss(reduction="mean")
    
    best_score = np.inf

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)
        
        # scoring
        score, scores = get_score(valid_labels, predictions)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}  Scores: {scores}')
        if CFG.wandb:
            wandb.log({f"[fold{fold}] epoch": epoch+1, 
                       f"[fold{fold}] avg_train_loss": avg_loss, 
                       f"[fold{fold}] avg_val_loss": avg_val_loss,
                       f"[fold{fold}] score": score})
        
        if best_score > score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")

    predictions = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    valid_folds[[f"pred_{c}" for c in CFG.target_cols]] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [17]:
if __name__ == '__main__':
    
    def get_result(oof_df):
        labels = oof_df[CFG.target_cols].values
        preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
        score, scores = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')
        return score
    
    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                score = get_result(_oof_df)
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        score = round(get_result(oof_df),3)
        oof_df.to_pickle(OUTPUT_DIR+f'oof_df.pkl')
        
    # if CFG.DEBUG:
    #     import send2trash
    #     send2trash.send2trash(OUTPUT_DIR)
    CFG.OUTPUT_DIR = OUTPUT_DIR
    dict_cfg = {k: vars(CFG)[k] for k in vars(CFG) if "__" not in k}
    with open(OUTPUT_DIR+"dict_cfg", 'wb') as web:
        pickle.dump(dict_cfg , web)
    with open(OUTPUT_DIR+"class_cfg", 'wb') as web:
        pickle.dump(CFG , web)
    
    
    if CFG.wandb:
        wandb.config.update(class2dict(CFG))
        wandb.finish()

========== fold: 0 training ==========
FunnelConfig {
  "_name_or_path": "/home/jupyter/models/funnel/intermediate/",
  "activation_dropout": 0.0,
  "architectures": [
    "FunnelModel"
  ],
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.0,
  "attention_type": "relative_shift",
  "block_repeats": [
    1,
    1,
    1
  ],
  "block_sizes": [
    6,
    6,
    6
  ],
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "hidden_act": "gelu_new",
  "hidden_dropout": 0.0,
  "hidden_dropout_prob": 0.0,
  "initializer_range": 0.1,
  "initializer_std": null,
  "layer_norm_eps": 1e-09,
  "max_position_embeddings": 512,
  "model_type": "funnel",
  "n_head": 12,
  "num_decoder_layers": 2,
  "output_hidden_states": true,
  "pool_q_only": true,
  "pooling_type": "mean",
  "rel_attn_type": "factorized",
  "separate_cls": true,
  "transformers_version": "4.21.2",
  "truncate_seq": true,
  "type_vocab_size": 3,
  "vocab_size": 30522
}



Epoch: [1][0/366] Elapsed 0m 1s (remain 11m 54s) Loss: 2.7511(2.7511) Grad: inf  LR: 0.00002000  
Epoch: [1][20/366] Elapsed 0m 14s (remain 3m 53s) Loss: 0.1820(0.8315) Grad: 150743.3281  LR: 0.00002000  
Epoch: [1][40/366] Elapsed 0m 26s (remain 3m 30s) Loss: 0.1831(0.5109) Grad: 90155.0000  LR: 0.00001998  
Epoch: [1][60/366] Elapsed 0m 38s (remain 3m 14s) Loss: 0.1073(0.3979) Grad: 134025.6719  LR: 0.00001996  
Epoch: [1][80/366] Elapsed 0m 51s (remain 3m 2s) Loss: 0.1394(0.3323) Grad: 96354.7812  LR: 0.00001993  
Epoch: [1][100/366] Elapsed 1m 4s (remain 2m 49s) Loss: 0.0970(0.2941) Grad: 87220.3594  LR: 0.00001990  
Epoch: [1][120/366] Elapsed 1m 17s (remain 2m 36s) Loss: 0.0514(0.2666) Grad: 78636.5703  LR: 0.00001985  
Epoch: [1][140/366] Elapsed 1m 29s (remain 2m 23s) Loss: 0.1472(0.2481) Grad: 142381.5781  LR: 0.00001980  
Epoch: [1][160/366] Elapsed 1m 42s (remain 2m 10s) Loss: 0.1073(0.2343) Grad: 173763.6250  LR: 0.00001974  
Epoch: [1][180/366] Elapsed 1m 55s (remain 1m 57

Epoch 1 - avg_train_loss: 0.1755  avg_val_loss: 0.1443  time: 279s
Epoch 1 - Score: 0.5376  Scores: [0.5095205567538782, 0.6686999920248118, 0.5441880746262279, 0.4898142142756081, 0.5234393638183819, 0.49005913439148924]
Epoch 1 - Save Best Score: 0.5376 Model


EVAL: [60/62] Elapsed 0m 46s (remain 0m 0s) Loss: 0.1444(0.1441) 
EVAL: [61/62] Elapsed 0m 46s (remain 0m 0s) Loss: 0.2232(0.1443) 
Epoch: [2][0/366] Elapsed 0m 0s (remain 5m 27s) Loss: 0.2552(0.2552) Grad: 475010.5625  LR: 0.00001866  
Epoch: [2][20/366] Elapsed 0m 13s (remain 3m 48s) Loss: 0.1052(0.1295) Grad: 205095.4375  LR: 0.00001851  
Epoch: [2][40/366] Elapsed 0m 26s (remain 3m 29s) Loss: 0.0803(0.1144) Grad: 115309.2891  LR: 0.00001836  
Epoch: [2][60/366] Elapsed 0m 39s (remain 3m 15s) Loss: 0.0869(0.1119) Grad: 230934.6719  LR: 0.00001820  
Epoch: [2][80/366] Elapsed 0m 51s (remain 3m 1s) Loss: 0.0929(0.1101) Grad: 254519.0938  LR: 0.00001803  
Epoch: [2][100/366] Elapsed 1m 4s (remain 2m 48s) Loss: 0.1200(0.1075) Grad: 142611.4688  LR: 0.00001786  
Epoch: [2][120/366] Elapsed 1m 16s (remain 2m 35s) Loss: 0.1154(0.1077) Grad: 287736.0625  LR: 0.00001768  
Epoch: [2][140/366] Elapsed 1m 29s (remain 2m 22s) Loss: 0.0796(0.1080) Grad: 136493.3906  LR: 0.00001749  
Epoch: [2][16

Epoch 2 - avg_train_loss: 0.1084  avg_val_loss: 0.1089  time: 278s
Epoch 2 - Score: 0.4678  Scores: [0.4962717931314254, 0.45680297741377285, 0.4428657373952123, 0.4615018374590605, 0.4938368237397075, 0.4554111499623152]
Epoch 2 - Save Best Score: 0.4678 Model


...

EVAL: [60/62] Elapsed 0m 46s (remain 0m 0s) Loss: 0.1070(0.1166) 
EVAL: [61/62] Elapsed 0m 46s (remain 0m 0s) Loss: 0.0559(0.1165) 
Epoch: [6][0/366] Elapsed 0m 0s (remain 5m 38s) Loss: 0.0462(0.0462) Grad: 128373.2031  LR: 0.00000135  
Epoch: [6][20/366] Elapsed 0m 13s (remain 3m 46s) Loss: 0.0639(0.0441) Grad: 171791.1875  LR: 0.00000121  
Epoch: [6][40/366] Elapsed 0m 26s (remain 3m 28s) Loss: 0.0310(0.0460) Grad: 57660.0000  LR: 0.00000108  
Epoch: [6][60/366] Elapsed 0m 38s (remain 3m 14s) Loss: 0.0579(0.0460) Grad: 98475.3516  LR: 0.00000095  
Epoch: [6][80/366] Elapsed 0m 51s (remain 3m 1s) Loss: 0.0539(0.0466) Grad: 134862.7031  LR: 0.00000083  
Epoch: [6][100/366] Elapsed 1m 4s (remain 2m 48s) Loss: 0.0393(0.0477) Grad: 67719.3203  LR: 0.00000072  
Epoch: [6][120/366] Elapsed 1m 16s (remain 2m 35s) Loss: 0.0606(0.0481) Grad: 128504.6562  LR: 0.00000062  
Epoch: [6][140/366] Elapsed 1m 29s (remain 2m 22s) Loss: 0.0551(0.0478) Grad: 172022.0312  LR: 0.00000053  
Epoch: [6][160/3

Epoch 6 - avg_train_loss: 0.0486  avg_val_loss: 0.1160  time: 278s
Epoch 6 - Score: 0.4829  Scores: [0.5252005172689923, 0.4835233141486791, 0.43574202063213135, 0.47249541684154767, 0.5060621076707432, 0.47454452301120753]


EVAL: [60/62] Elapsed 0m 46s (remain 0m 0s) Loss: 0.1069(0.1161) 
EVAL: [61/62] Elapsed 0m 46s (remain 0m 0s) Loss: 0.0512(0.1160) 


========== fold: 3 result ==========
Score: 0.4630  Scores: [0.495473454968675, 0.45947201574197266, 0.4247375379993438, 0.45392891192096657, 0.4789833095403754, 0.46529390662873654]
========== CV ==========
Score: 0.4645  Scores: [0.4938098460543921, 0.46181244790089493, 0.42338075198399694, 0.46028171829402215, 0.47987051448913387, 0.46793835834106373]


[fold0] avg_train_loss,█▃▃▂▁▁
[fold0] avg_val_loss,█▂▂▁▁▁
[fold0] epoch,▁▂▄▅▇█
[fold0] loss,▇█▅▅▆▃█▆▄▂▆▂▃▅▄▅▂▄▂▅▁▃▄▃▃▂▂▄▂▄▂▃▃▂▃▂▂▁▂▄
[fold0] lr,███████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
[fold0] score,█▂▂▁▁▁
[fold1] avg_train_loss,█▄▃▂▁▁
[fold1] avg_val_loss,▇▃█▁▂▁
[fold1] epoch,▁▂▄▅▇█
[fold1] loss,▆▆▇▅▆▄▄▄▇▆█▅▅▁▄▆▂▄▄▃▂▄▂▃▃▁▄▂▂▄▃▁▃▂▄▃▂▄▄▂
[fold1] lr,███████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁


In [18]:
print(OUTPUT_DIR)

/home/jupyter/output/ex/funnel-intermediate/001/202210311829/


In [19]:
# import os
# OUTPUT_DIR = "/home/jupyter/output/ex/roberta-base/004/202210151736/"
# PREDICT_DIR = OUTPUT_DIR.replace("output","predict")
# if not os.path.exists(PREDICT_DIR):
#     os.makedirs(PREDICT_DIR)


In [20]:
if CFG.TO_KAGGLE:
    UPLOAD_DIR = OUTPUT_DIR
    EX_NO = f"{model_name}-{CFG.file_name}" # 実験番号などを入れる、folderのpathにする
    USERID = 'your_id'


    def dataset_upload():
        import json
        from kaggle.api.kaggle_api_extended import KaggleApi

        id = f'{USERID}/{EX_NO}'

        dataset_metadata = {}
        dataset_metadata['id'] = id
        dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
        dataset_metadata['title'] = f'{EX_NO}'

        with open(UPLOAD_DIR +'dataset-metadata.json', 'w') as f:
            json.dump(dataset_metadata, f, indent=4)

        api = KaggleApi()
        api.authenticate()

        # データセットがない場合
        if f'{USERID}/{EX_NO}' not in [str(d) for d in api.dataset_list(user=USERID, search=f'"{EX_NO}"')]:
            api.dataset_create_new(folder=UPLOAD_DIR,
                                   convert_to_csv=False,
                                   dir_mode='skip')
            
            
             #フォルダーを削除
            if f'{USERID}/{EX_NO}' not in [str(d) for d in api.dataset_list(user=USERID, search=f'"{EX_NO}"')]:
                remove_files = glob.glob(OUTPUT_DIR+"*")
                remove_files.remove(OUTPUT_DIR+"oof_df.pkl")
                for file in remove_files:
                    os.remove(file)
                print("folder upload")
                            #apiコマンドを書き込む
                f = open(f'{model_name}_api_command.txt', 'a')
                api_command = f"!kaggle datasets download -d hiroki8383/{EX_NO}\n"
                f.write(api_command)
                f.close()
            else:
                print("folder not upload")
            
            
        # データセットがある場合→更新されない場合がある（後で原因追及)
        else:
            print("this folder exsits")
            # api.dataset_create_version(folder=UPLOAD_DIR,
            #                            version_notes='update',
            #                            convert_to_csv=False,
            #                            delete_old_versions=False,
            #                            dir_mode='zip')

        

        
    dataset_upload()

Starting upload for file config.pth


100% 2.55k/2.55k [00:02<00:00, 1.20kB/s]


Upload successful: config.pth (3KB)
Starting upload for file funnel-intermediate_fold2_best.pth


100% 676M/676M [00:19<00:00, 36.7MB/s] 


Upload successful: funnel-intermediate_fold2_best.pth (676MB)
Starting upload for file class_cfg


...

Upload successful: train.log (10KB)
Starting upload for file funnel-intermediate_fold0_best.pth


100% 676M/676M [00:25<00:00, 27.6MB/s] 


Upload successful: funnel-intermediate_fold0_best.pth (676MB)
Starting upload for file dict_cfg


100% 458k/458k [00:02<00:00, 185kB/s]


Upload successful: dict_cfg (458KB)
folder upload


In [22]:
if not CFG.DEBUG:
    def to_write_score(CFG):
        df = pd.read_csv(CFG.score_path)
        def get_result2(oof_df):
                labels = oof_df[CFG.target_cols].values
                preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
                score, scores = get_score(labels, preds)
                LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')
                return score,scores

        score,scores = get_result2(oof_df)
        name = "-".join(OUTPUT_DIR.split("/")[-4:-2])
        base = {"name":name,"score":score,"memo":CFG.MEMO} 
        base.update(dict(zip(CFG.target_cols,scores)))
        df = df.append(base,ignore_index=True)
        df.to_csv(CFG.score_path,index=False)
    to_write_score(CFG)

Score: 0.4645  Scores: [0.4938098460543921, 0.46181244790089493, 0.42338075198399694, 0.46028171829402215, 0.47987051448913387, 0.46793835834106373]


In [23]:
df = pd.read_csv(CFG.score_path)
df

,name,memo,LB,score,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,base-discriminator-001,NaN,0.46,0.466957,0.501431,0.455460,0.428477,0.465690,0.485904,0.464780
1,base-discriminator-002,NaN,0.45,0.474078,0.504413,0.462707,0.432588,0.469546,0.500918,0.474296
2,deberta-v3-base-003,2epochs,NaN,0.459342,0.496531,0.450482,0.419448,0.457413,0.476347,0.455832
3,deberta-v3-base-003,6epochs,0.44,0.456879,0.488333,0.448687,0.417134,0.458769,0.479633,0.448721
4,deberta-v3-base-006,15folds,NaN,0.463635,0.479650,0.475910,0.442828,0.463557,0.456412,0.463457
5,deberta-v3-base-006,20folds,NaN,0.449212,0.475585,0.433808,0.415852,0.457436,0.484686,0.427907
6,deberta-v3-base-006,10folds,NaN,0.446338,0.490661,0.435001,0.409460,0.417236,0.481800,0.443871
7,distilroberta-base-001,4folds,NaN,0.470528,0.501629,0.459765,0.426740,0.473205,0.501976,0.459850
8,distilroberta-base-002,"バッチ16,4folds",NaN,0.471035,0.503345,0.458993,0.427209,0.471235,0.504850,0.460580
9,microsoft-deberta-v3-large-001,NaN,0.46,0.450311,0.482961,0.445596,0.410873,0.449346,0.466162,0.446928
